In [1]:
import cv2
from glob import glob
from ultralytics import YOLO
from tqdm import tqdm
from typing import List

import torch.nn as nn
import torch
import os
import math
import csv

## 이전 파일 클래스 정의

In [2]:
from torchvision.models import resnet50
class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.base_model = resnet50(pretrained=True)
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

In [3]:
class TwoPointCoordinate:
    def __init__(self, start_point: tuple, end_point: tuple):
        self.start_point = start_point
        self.end_point = end_point

    def get_xy_coords(self):
        x1, y1 = self.start_point[1], self.start_point[0]  # x1 = x4, x2 = x3
        x3, y3 = self.end_point[1], self.end_point[0]   # y1 = y2, y3 = y4
        # return [x1, y1, x3, y1, x3, y3, x1, y3]
        return (y1, x1), (y3, x3)

    def get_csv_format(self):
        x1, y1 = self.start_point[1], self.start_point[0]  # x1 = x4, x2 = x3
        x3, y3 = self.end_point[1], self.end_point[0]  # y1 = y2, y3 = y4
        return [x1, y1, x3, y1, x3, y3, x1, y3]


class YoloCoordinate:
    def __init__(self, w, h, center_x, center_y):
        self.w = w
        self.h = h
        self.center_x = center_x
        self.center_y = center_y

    def convert_two_point_coordinate(self) -> TwoPointCoordinate:
        half_width = self.w / 2
        half_height = self.h / 2
        img_width = 1920
        img_height = 1040

        point1_x = self.center_x - half_width # box의 왼쪽 위 꼭지점
        point1_y = self.center_y - half_height # box의 왼쪽 위 꼭지점

        point3_x = self.center_x + half_width # box의 오른쪽 아래 꼭지점
        point3_y = self.center_y + half_height # box의 오른쪽 아래 꼭지점

        point1_x = point1_x * img_width
        point1_y = point1_y * img_height

        point3_x = point3_x * img_width
        point3_y = point3_y * img_height

        return TwoPointCoordinate((point1_y, point1_x), (point3_y, point3_x))

In [4]:
# 파일 이름, 라벨, 좌표 클래스 (TwoPoints.., Yolo..)를 담는 클래스
class ImageContainer:
    def __init__(self, file_names : List[str], labels: List[str], coordinates: List, confidences: List, cropped_image: List):
        self.coordinates = coordinates
        self.file_names = file_names
        self.labels = labels
        self.confidences = confidences
        self.cropped_image = cropped_image
        self.convert_coordinates()

    def convert_coordinates(self):
        if isinstance(self.coordinates[0], YoloCoordinate):
            for idx, coordinate in tqdm(enumerate(self.coordinates), desc='TwoPointCoordinate로 변환'):
                self.coordinates[idx] = coordinate.convert_two_point_coordinate()

    def remove(self, idx):
        self.coordinates.pop(idx)
        self.file_names.pop(idx)
        self.labels.pop(idx)
        self.confidences.pop(idx)
        self.cropped_image.pop(idx)

    def to_csv(self, file_path="data.csv"):
        coord_li = []
        for coord in self.coordinates:
            coord_li.append(coord.get_csv_format())
        result = list(zip(self.file_names, self.labels, self.confidences, coord_li))
        flattened = [flatten(sublist) for sublist in result]
        self.__write(flattened, file_path)

    def __write(self, flattened, file_path):
        headers = [
            "file_name", "class_id", "confidence",
            "point1_x", "point1_y", "point2_x", "point2_y",
            "point3_x", "point3_y", "point4_x", "point4_y"
        ]

        # CSV 파일에 데이터 추가
        with open(file_path, "a", newline="") as file:
            writer = csv.writer(file)

            # 헤더 작성
            if file.tell() == 0:
                writer.writerow(headers)

            # 데이터 작성
            for data in flattened:
                writer.writerow(data)
            print(f"{len(flattened)}개의 데이터가 저장되었습니다.")

In [5]:
# 이미지 컨테이너에 담긴 이미지 정보를 토대로 좌표에 맞추어 자르고 저장하는 클래스

class LightImageCropper: # 자를려면 자를 이미지, 이미지 좌표
    @staticmethod
    def crop(image, coordinates):
        cropped_images = []

        for idx, coordinate in enumerate(coordinates):
            y_1 = int(coordinate.start_point[0])
            x_1 = int(coordinate.start_point[1])
            y_2 = int(coordinate.end_point[0])
            x_2 = int(coordinate.end_point[1])
            cropped_images.append(image[y_1: y_2, x_1: x_2, :].copy())
        return cropped_images


## 신규 클래스 ModelParser
모델을 입력받아서 ImageContainer로 저장하는 클래스

In [6]:
class ModelParser:
    def __init__(self, model, img_paths):
        self.model = model
        self.img_paths = img_paths
        self.detections = { "names": [], "labels": [], "boxes": [], "confs": [], "cropped_images": []}

    def parse(self) -> ImageContainer:
        for path in self.img_paths:
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            filename = os.path.basename(path)
            self.detect(img, filename)

        return ImageContainer(self.detections["names"], self.detections["labels"], self.detections["boxes"], self.detections["confs"], self.detections["cropped_images"])

    # TODO 가로등과 같은 이미지를 제외하기

    def detect(self, img, filename): # 욜로 모델에서 이미지를 불러와서 탐지한 후 json으로 변환
        detection = self.model(img)[0]

        confs = detection.boxes.conf.tolist()
        names = [filename for _ in range(len(confs))]
        labels = detection.boxes.cls.int().tolist()
        coordinates = self.__parse_coordinates(detection.boxes.xyxy.tolist())

        detect_dict = {
            "names": names,
            "labels": labels,
            "boxes": coordinates,
            "confs": confs,
        }

        # detect_dict = self.__remove_overlap_coords(detect_dict)
        self.__add_detection(detect_dict)
        self.detections["cropped_images"] += LightImageCropper.crop(img, detect_dict['boxes'])


    def __add_detection(self, detect_dict):
        for key in detect_dict.keys():
            self.detections[key] += detect_dict[key]

    def __parse_coordinates(self, coordinates):
        coord_li = []
        for coor in coordinates:
            coord_li.append(self.__trans_coordinate(coor))
        return coord_li

    def __trans_coordinate(self, coordinate):

        point1_y, point1_x = coordinate[1], coordinate[0]
        point3_y, point3_x = coordinate[3], coordinate[2]

        start_point, end_point = (point1_y, point1_x), (point3_y, point3_x)

        return TwoPointCoordinate(start_point, end_point)

    def __remove_overlap_coords(self, detect_dict):
        boxes = detect_dict['boxes']
        confs = detect_dict['confs']
        overlap_idx = self.__validation(boxes, confs)

        if overlap_idx == -1:
            return detect_dict

        for key in detect_dict.keys():
            detect_dict[key].pop(overlap_idx)
        return detect_dict


    def __validation(self, boxes, confs:List): # 원소는 TwoPointCoordinate
        for i in range(len(boxes)):
            coords1 = boxes[i].get_xy_coords()
            conf1 = confs[i]

            for j in range(i + 1, len(boxes)):
                coords2 = boxes[j].get_xy_coords()
                conf2 = confs[j]

                y1, x1 = coords1[0]  # rect1의 좌상단 좌표
                y2, x2 = coords1[1]  # rect1의 우하단 좌표
                y3, x3 = coords2[0]  # rect2의 좌상단 좌표
                y4, x4 = coords2[1]  # rect2의 우하단 좌표

                if x2 <= x3 or x4 <= x1:  # x축 범위가 겹치지 않는 경우
                    continue
                if y2 <= y3 or y4 <= y1:  # y축 범위가 겹치지 않는 경우
                    continue

                return confs.index(min(conf1, conf2))
        return -1



## ImageClassifier, CsvWriter
ResNet 모델을 입력받아서 잘린 이미지를 분류하는 클래스
Csv파일로 변환하는 클래스

In [7]:
from PIL import Image
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor()
])
class_dict = {0: '0',
 1: '1',
 2: '10',
 3: '11',
 4: '12',
 5: '13',
 6: '14',
 7: '15',
 8: '16',
 9: '17',
 10: '18',
 11: '19',
 12: '2',
 13: '20',
 14: '21',
 15: '22',
 16: '23',
 17: '24',
 18: '25',
 19: '26',
 20: '27',
 21: '28',
 22: '29',
 23: '3',
 24: '30',
 25: '31',
 26: '32',
 27: '33',
 28: '4',
 29: '5',
 30: '6',
 31: '7',
 32: '8',
 33: '9'}

# 모델 예측
import matplotlib.pyplot as plt
class ImageClassifier:
    def __init__(self, model):
        self.model = model

    def classify(self, cutting_image):
        with torch.no_grad():
            cutting_image = cutting_image.to(device)
            self.model.eval()
            output = self.model(cutting_image.unsqueeze(0))  # 배치 차원 추가
            _, predicted_class = torch.max(output, 1)

        # return predicted_class.item()
        return class_dict[predicted_class.item()]
    def run_classifier(self, container): # 잘린 이미지들을 받아서 레스넷 돌리는 함수
        count = 0
        cropped_images = container.cropped_image
        for idx, crop_img in tqdm(enumerate(cropped_images), leave=False, position=0, desc='Processing images'):
            # if not self.__validation(crop_img):
                # import uuid
                # # 고유한 파일 이름 생성
                # unique_filename = str(uuid.uuid4()) + ".png"
                #
                # # 이미지 파일로 저장
                # output_path = "./temp/" +unique_filename
                # cv2.imwrite(output_path, crop_img)
                #
                # container.remove(idx)
                # continue


            # OpenCV는 BGR 형식으로 이미지를 불러오므로 RGB 형식으로 변환
            image = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)

            # PIL Image로 변환
            image = transform(Image.fromarray(image))

            label = self.classify(image)
            container.labels[idx] = label

    @staticmethod
    def __validation(image):
        height, width, _ = image.shape
        return height >= 150 and width >= 150



## 경로변수 설정

In [8]:
# 경로 변수
model_path = f'./model/yolo/best.pt'  # best.pt 파일이 있는 경로
model = YOLO(model_path)

# resNetModel = ResNet50(34)
# resNetModel.load_state_dict(torch.load("./model/Resnet50augmented.pt"))
resNetModel = torch.load("./Resnet50augmented(temp).pt")

In [9]:
test_image_paths = glob("../detection/datasets/test/*")

len(test_image_paths)

3400

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

def calc_batch_range(process, batch_size):
    return process, process + batch_size

def flatten(lst):
    result = []
    for i in lst:
        if isinstance(i, list):
            result.extend(flatten(i))
        else:
            result.append(i)
    return result

# RUN!!

배치 사이즈는 메모리 상태보고 조절하시면 됩니다 32GB기준으로 1000정도가 적당합니다

In [11]:
# 메모리 상태 보시고 적절히 조절해주세요
batch_size = 700
process = 0
trials = math.ceil(len(test_image_paths) / batch_size)


for _ in tqdm(range(trials)):
    # 배치 사이즈 크기만큼 자르기
    start, end = calc_batch_range(process, batch_size)

    # 모델에서 좌표값 읽어오기
    # ModelParser에서 이미지를 걸러냅니다
    model_parser = ModelParser(model, test_image_paths[start: end])
    container: ImageContainer = model_parser.parse()

    # 이미지 분류하기
    labels = ImageClassifier(resNetModel).run_classifier(container)

    # csv파일 저장하기
    container.to_csv()

    process += batch_size


  0%|          | 0/5 [00:00<?, ?it/s]
0: 384x640 1 genesis_gv80_suv_2020_, 61.4ms
Speed: 3.0ms preprocess, 61.4ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 genesis_gv80_suv_2020_, 1 kia_carnival_van_2015_2020, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_trailblazer_suv_2021_, 1 genesis_gv80_suv_2020_, 7.3ms
Speed: 2.0ms preprocess, 7.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 hyundai_ioniq_hatchback_2016_2019, 8.8ms
Speed: 2.0ms preprocess, 8.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 hyundai_avante_sedan_2011_2015, 9.0ms
Speed: 1.5ms preprocess, 9.0ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 kia_carnival_van_2015_2020, 7.1ms
Speed: 2.0ms preprocess, 7.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 ssangyong_tivoli_su

1189개의 데이터가 저장되었습니다.


0: 384x640 1 hyundai_avante_sedan_2011_2015, 7.0ms
Speed: 2.1ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 genesis_gv80_suv_2020_, 6.9ms
Speed: 1.0ms preprocess, 6.9ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 kia_stonic_suv_2017_2019, 1 renault_sm3_sedan_2015_2018, 7.1ms
Speed: 2.0ms preprocess, 7.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 hyundai_avante_sedan_2020_, 1 hyundai_ioniq_hatchback_2016_2019, 8.4ms
Speed: 1.0ms preprocess, 8.4ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 genesis_g80_sedan_2021_, 1 genesis_gv80_suv_2020_, 8.3ms
Speed: 1.0ms preprocess, 8.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 genesis_gv80_suv_2020_, 8.9ms
Spee

1520개의 데이터가 저장되었습니다.


Speed: 2.5ms preprocess, 8.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 kia_carnival_van_2021_, 9.5ms
Speed: 1.0ms preprocess, 9.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 kia_morning_hatchback_2011_2016, 9.0ms
Speed: 2.1ms preprocess, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 chevrolet_malibu_sedan_2017_2019s, 1 genesis_g80_sedan_2021_, 8.5ms
Speed: 1.5ms preprocess, 8.5ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 genesis_g80_sedan_2021_, 8.2ms
Speed: 1.4ms preprocess, 8.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 genesis_g80_sedan_2021_, 1 kia_sportage_suv_2016_2020, 9.4ms
Speed: 2.0ms preprocess, 9.4ms inference, 3.6ms postprocess per image at shap

1525개의 데이터가 저장되었습니다.


0: 384x640 1 genesis_g80_sedan_2016_2020, 1 hyundai_ioniq_hatchback_2016_2019, 1 kia_morning_hatchback_2004_2010, 12.3ms
Speed: 2.0ms preprocess, 12.3ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 ssangyong_korando_suv_2019_2020, 7.1ms
Speed: 3.0ms preprocess, 7.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 kia_morning_hatchback_2004_2010, 1 renault_xm3_suv_2020_, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 kia_morning_hatchback_2004_2010, 1 kia_ray_hatchback_2012_2017, 1 renault_xm3_suv_2020_, 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_2017_2019, 1 kia_ray_hatchback_2012_2017, 7.9ms
Speed: 2.0ms preprocess, 7.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_malibu_sedan_

1471개의 데이터가 저장되었습니다.


0: 384x640 1 chevrolet_malibu_sedan_2012_2016, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 renault_xm3_suv_2020_, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 chevrolet_trax_suv_2017_2019, 1 hyundai_avante_sedan_2020_, 7.0ms
Speed: 1.5ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 hyundai_avante_sedan_2020_, 1 hyundai_grandeur_sedan_2011_2016, 1 hyundai_sonata_sedan_2019_2020, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 hyundai_grandstarex_van_2018_2020, 1 kia_stonic_suv_2017_2019, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 genesis_g80_sedan_2016_2020, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640

1206개의 데이터가 저장되었습니다.
